# Get city statistics from climate data downloaded from CDS
Workflow to get day- and nighttime utci statistics

In [ ]:
import sqlite3
import pandas as pd
import xarray as xr
from src.utils import *
input_folder = "./../../../s3/data/d003_climate/cl_01_utci/raw_data/"


## Download hourly data

In [ ]:
year = 1991
filename = CDS_get_utci_hourly_zipped(input_folder, year)
output_folder = unzip_to_folder(input_folder, file_name)

## Get city coordinates

In [2]:
# path to databases
city_geom  = './../../../s3/data/d000_lookuptables/city_pts_urban_audit2021.sqlite'
con = sqlite3.connect(city_geom)
# read full table
city_all = pd.read_sql_query("SELECT _wgs84x, _wgs84y, city_code FROM urau_lb_2021_3035_cities_center_points_4", con)
con.close()
# get city coordinates
# lonlat_list =[["NL005C", 4.640960, 52.113299], ["NL006C", 5.384670, 52.173656], ["NL007C", 5.921886, 52.189884]]
lon_list = city_all["_wgs84x"].values.tolist()
lat_list = city_all["_wgs84y"].values.tolist()
city_list = city_all["city_code"].values.tolist()
target_lon = xr.DataArray(lon_list, dims="city", coords={"city": city_list})
target_lat = xr.DataArray(lat_list, dims="city", coords={"city": city_list})

In [3]:
target_lat

<xarray.DataArray (city: 729)>
array([ 52.11329906,  52.17365631,  52.18988429,  51.99798033,
        53.00127959,  51.50348208,  44.91089281,  38.11277533,
        40.8596404 ,  51.58512555,  52.26823719,  52.06693717,
        51.44687419,  52.22080743,  43.4450878 ,  42.76727575,
        41.11272142,  41.18278877,  52.01528463,  52.05978302,
        53.21862867,  52.40493016,  52.3080715 ,  40.60615979,
        40.74465475,  39.22501475,  40.59430694,  41.2051568 ,
        44.80159458,  52.50804172,  50.89977969,  51.47615239,
        52.26630769,  51.71678213,  52.2233555 ,  52.6298811 ,
        37.91384906,  45.40337724,  45.53257786,  41.27834807,
        53.1700065 ,  52.16511694,  52.54278164,  50.85297336,
        44.64388255,  37.09418349,  38.07624737,  41.50193135,
        51.48727711,  51.83860281,  51.83620065,  51.78153928,
        52.50213845,  36.90702051,  41.15727938,  41.4566493 ,
        44.53340449,  38.19112563,  51.92293769,  51.51318373,
        50.99701746,  52.17599351,  41.23913318,  40.68919934,
        45.02979754,  46.49224842,  43.87814185,  44.79366909,
        51.57357171,  52.07124204,  52.02363829,  51.42003917,
        52.46298233,  52.06088387,  52.51869635,  46.06406366,
        44.10128061,  43.52400637,  45.48325719,  45.44870176,
...
        41.54447825,  47.06628329,  46.56487253,  46.17462421,
        51.12769331,  50.94613218,  52.52781736,  53.33434958,
        51.64701159,  49.78057533,  32.67884502,  40.68803586,
        52.40645916,  50.72082891,  51.13587695,  51.65936451,
        51.66154841,  50.56207536,  51.53377465,  54.35919866,
        53.17434589,  51.84292922,  50.84416631,  51.85802977,
        54.08952159,  53.82135124,  40.21615491,  41.4608325 ,
        53.4318025 ,  54.50644516,  50.27617806,  41.71701864,
        53.13129033,  51.22575873,  50.29923261,  50.31564821,
        38.51073874,  44.03883415,  41.46641293,  45.54494971,
        43.13646954,  43.6723307 ,  45.66381329,  45.82404205,
        42.39612918,  59.98177108,  43.57271878,  42.45588594,
        44.91326132,  45.19770356,  54.69257123,  54.89931594,
        41.56458593,  41.08743974,  44.07706676,  39.27381871,
        44.33611145,  55.73041663,  54.40108507,  55.70928043,
        55.90499687,  40.66969292,  37.63435521,  49.61357853,
        35.89434514,  52.35245505,  52.61846432,  42.57957971,
        45.43959583,  40.47437586,  40.65926182,  38.88987171,
        40.91382078,  45.95383892,  45.85760622,  52.34792185,
        52.39636978])
Coordinates:
  * city     (city) <U6 'NL005C' 'NL006C' 'NL007C' ... 'NL003C' 'NL004C'

## Read the downloaded .nc file with xarray

In [5]:
climate_path = output_folder+"ECMWF_utci_2018010*.nc"
data = xr.open_mfdataset(climate_path , engine="netcdf4", parallel=True)

## Compute statistics

In [6]:
data_cities = data["utci"].sel(lon=target_lon, lat=target_lat, method="nearest")
data_cities_daytime = data_cities.resample(time="12H", base = 7)
utci_mean = data_cities_daytime.mean()
utci_min = data_cities_daytime.min()
utci_max = data_cities_daytime.max()
stats = xr.concat([utci_mean, utci_min, utci_max], dim="stats").assign_coords(stats=["mean", "min", "max"])
stats_df = stats.to_dataframe()

<xarray.DataArray 'utci' (stats: 3, time: 19, city: 729)>
dask.array<concatenate, shape=(3, 19, 729), dtype=float32, chunksize=(1, 1, 729), chunktype=numpy.ndarray>
Coordinates:
    lon      (city) float64 4.75 5.5 6.0 6.0 6.5 ... 14.75 12.75 9.5 6.75 5.25
    lat      (city) float64 52.0 52.25 52.25 52.0 53.0 ... 46.0 45.75 52.25 52.5
  * city     (city) <U6 'NL005C' 'NL006C' 'NL007C' ... 'NL003C' 'NL004C'
  * time     (time) datetime64[ns] 2017-12-31T19:00:00 ... 2018-01-09T19:00:00
  * stats    (stats) <U4 'mean' 'min' 'max'
Attributes:
    code:     167
    table:    128

In [7]:
stats_daytime = stats.sel(time=(stats.time.dt.hour == 7))
stats_nighttime = stats.sel(time=(stats.time.dt.hour == 19))

In [8]:
stats_daytime.to_dataframe()

lon    lat        utci
stats time                city                            
mean  2018-01-01 07:00:00 NL005C   4.75  52.00  268.353119
                          NL006C   5.50  52.25  268.309174
                          NL007C   6.00  52.25  268.205078
                          NL008C   6.00  52.00  268.205078
                          NL009C   6.50  53.00  265.685242
...                                 ...    ...         ...
max   2018-01-09 07:00:00 IT050C  14.75  41.00  289.999786
                          IT051C  12.75  46.00  282.637970
                          IT052C   9.50  45.75  281.866119
                          NL003C   6.75  52.25  278.168121
                          NL004C   5.25  52.50  277.283844

[19683 rows x 3 columns]